In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0,"../")

In [ ]:
# default_exp model.model

In [ ]:
#export

import torch
import torch.nn as nn
from random import sample
import torch.nn as nn
import torch

from core.model.encoder import Encoder
from core.model.utils import pair_embeddings, stack_features_across_batch, convert_indices

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import Dataset, DataLoader
from core.dataloader import CLEVR_train, collate_boxes, CLEVR_train_onlyquery, collate_boxes_onlyquery
from core.model.utils import pair_embeddings, stack_features_across_batch

In [ ]:
train_dataset = CLEVR_train(root_dir='/home/mprabhud/dataset/clevr_lang/npys/ab_5t.txt', hyp_N=2)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True, collate_fn=collate_boxes)

Initialised..... 234  files...


In [ ]:
for b in train_loader:
    feed_dict_q_, feed_dict_k_, metadata_ = b
    break

In [ ]:
feed_dict_q_["images"] = feed_dict_q_["images"].cuda()
feed_dict_k_["images"] = feed_dict_k_["images"].cuda()

In [ ]:
#export
class MoCo_scene_and_view(nn.Module):
    """
    Build a MoCo model with: a query encoder, a key encoder, and a queue
    https://arxiv.org/abs/1911.05722
    """
    def __init__(self, dim=256, scene_r=35, view_r = 40, m=0.999, T=0.1, mlp=False, mode=None):
        """
        dim: feature dimension (default: 128)
        r: queue size; number of negative samples/prototypes (default: 16384)
        m: momentum for updating key encoder (default: 0.999)
        T: softmax temperature 
        mlp: whether to use mlp projection
        """
        super(MoCo_scene_and_view, self).__init__()

        self.scene_r = scene_r
        self.view_r = view_r
        self.m = m
        self.T = T
        self.mode = mode


        self.encoder_q = Encoder(dim = dim, mode=self.mode)
        self.encoder_k = Encoder(dim = dim, mode=self.mode)

        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data.copy_(param_q.data)  # initialize
            param_k.requires_grad = False  # not update by gradient

        # create the scene queue
        self.register_buffer("queue_scene", torch.randn(dim, scene_r))
        self.queue_scene = nn.functional.normalize(self.queue_scene, dim=0)
        self.register_buffer("queue_scene_ptr", torch.zeros(1, dtype=torch.long))
        
        # create the view queue
        self.register_buffer("queue_view", torch.randn(dim, view_r))
        self.queue_view = nn.functional.normalize(self.queue_view, dim=0)
        self.register_buffer("queue_view_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """
        Momentum update of the key encoder
        """
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1. - self.m)

    @torch.no_grad()
    def _dequeue_and_enqueue_scene(self, keys):

        batch_size = keys.shape[0]

        ptr = int(self.queue_scene_ptr)
        #removes for now
#         assert self.r % batch_size == 0  # for simplicity

        # replace the keys at ptr (dequeue and enqueue)
        self.queue_scene[:, ptr:ptr + batch_size] = keys.T
        ptr = (ptr + batch_size) % self.scene_r  # move pointer
        
        if self.scene_r % batch_size != 0:
            ptr=0

        self.queue_scene_ptr[0] = ptr
        
    @torch.no_grad()
    def _dequeue_and_enqueue_view(self, keys):

        batch_size = keys.shape[0]

        ptr = int(self.queue_view_ptr)
        #removes for now
#         assert self.r % batch_size == 0  # for simplicity

#         if ptr + batch_size>=self.view_r:
#             ptr=0

        # replace the keys at ptr (dequeue and enqueue)
        self.queue_view[:, ptr:ptr + batch_size] = keys.T
        ptr = (ptr + batch_size) % self.view_r  # move pointer
        
        self.queue_view_ptr[0] = ptr

    def forward(self, feed_dict_q, feed_dict_k=None, metadata=None, is_eval=False, cluster_result=None, index=None):
        """
        Input:
            feed_dict_q: a batch of query images and bounding boxes
            feed_dict_k: a batch of key images and bounding boxes
            is_eval: return momentum embeddings (used for clustering)
            cluster_result: cluster assignments, centroids, and density
            index: indices for training samples
            mode : should be either 'node' or 'spatial' depending on whether training for node or spatial embeddings
        Output:
            logits, targets, proto_logits, proto_targets
        """
        
        mode = self.mode
        hyp_N = feed_dict_q["objects"][0].item()
        
        if mode=="node":
            rel_viewpoint=None    
        
        if is_eval:
            # the output from encoder is a list of features from the batch where each batch element (image)
            # might contain different number of objects
            k = self.encoder_k(feed_dict_q) 
            
            # encoder output features in the list are stacked to form a tensor of features across the batch
            k = stack_features_across_batch(k, mode)
            
            # normalize feature across the batch
            k = nn.functional.normalize(k, dim=1)            
            return k
        
        rel_viewpoint = metadata["rel_viewpoint"]
        
        if mode=="node":
            rel_viewpoint=None
        
        # compute key features
        with torch.no_grad():  # no gradient to keys
            self._momentum_update_key_encoder()  # update the key encoder
            
            k_outputs = self.encoder_k(feed_dict_k, rel_viewpoint)   


        # compute query features
        q_outputs = self.encoder_q(feed_dict_q)  # queries: NxC
        
#         k,q = pair_embeddings(k_outputs, q_outputs, mode)
        k,q = k_outputs, q_outputs
        
        k = stack_features_across_batch(k, mode)
        q = stack_features_across_batch(q, mode)
        
        q = nn.functional.normalize(q, dim=1)
        
        with torch.no_grad():
            k = nn.functional.normalize(k, dim=1)
            
        # compute logits
        # Einstein sum is more intuitive
        # positive logits: Nx1
        l_pos = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)
        # negative logits: Nxr
        l_neg = torch.einsum('nc,ck->nk', [q, self.queue_scene.clone().detach()])

        # logits: Nx(1+r)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.T

        # labels: positive key indicators
        labels = torch.zeros(logits.shape[0], dtype=torch.long).cuda()

        # dequeue and enqueue
        self._dequeue_and_enqueue_scene(k)
        
        index = convert_indices(index,hyp_N, mode)
        
        # prototypical contrast
        if cluster_result is not None:  
            proto_labels = []
            proto_logits = []
            for n, (im2cluster,prototypes,density) in enumerate(zip(cluster_result['im2cluster'],cluster_result['centroids'],cluster_result['density'])):
                # get positive prototypes
                pos_proto_id = im2cluster[index]
                pos_prototypes = prototypes[pos_proto_id]    
                
                # sample negative prototypes
                all_proto_id = [i for i in range(im2cluster.max())] 
                
                #print(len(pos_prototypes), len(all_proto_id))
                neg_proto_id = set(all_proto_id)-set(pos_proto_id.tolist())
                neg_proto_id = sample(neg_proto_id,self.scene_r) #sample r negative prototypes 
                neg_prototypes = prototypes[neg_proto_id]    

                proto_selected = torch.cat([pos_prototypes,neg_prototypes],dim=0)
                
                # compute prototypical logits
                logits_proto = torch.mm(q,proto_selected.t())
                
                # targets for prototype assignment
                labels_proto = torch.linspace(0, q.size(0)-1, steps=q.size(0)).long().cuda()
                
                # scaling temperatures for the selected prototypes
                temp_proto = density[torch.cat([pos_proto_id,torch.LongTensor(neg_proto_id).cuda()],dim=0)]  
                logits_proto /= temp_proto
                
                proto_labels.append(labels_proto)
                proto_logits.append(logits_proto)
            return logits, labels, proto_logits, proto_labels
        else:
            return logits, labels, None, None
        
        
    def view_forward(self, feed_dict_q, feed_dict_k=None, metadata=None, feed_dicts_N=None, is_eval=False):
        """
        Input:
            feed_dict_q: a batch of query images and bounding boxes
            feed_dict_k: a batch of key images and bounding boxes
            is_eval: return momentum embeddings (used for clustering)
            cluster_result: cluster assignments, centroids, and density
            index: indices for training samples
            mode : should be either 'node' or 'spatial' depending on whether training for node or spatial embeddings
        Output:
            logits, targets, proto_logits, proto_targets
        """
        mode = self.mode
        hyp_N = feed_dict_q["objects"][0].item()
        
        if mode=="node":
            rel_viewpoint=None    
        
        if is_eval:
            # the output from encoder is a list of features from the batch where each batch element (image)
            # might contain different number of objects
            k = self.encoder_k(feed_dict_q) 
            
            # encoder output features in the list are stacked to form a tensor of features across the batch
            k = stack_features_across_batch(k, mode)
            
            # normalize feature across the batch
            k = nn.functional.normalize(k, dim=1)            
            return k
        
        rel_viewpoint = metadata["rel_viewpoint"]
        
        if mode=="node":
            rel_viewpoint=None
        
        # compute key features
        with torch.no_grad():  # no gradient to keys
            self._momentum_update_key_encoder()  # update the key encoder
            
            #compute key features
            k_outputs = self.encoder_k(feed_dict_k, rel_viewpoint)


        # compute query features
        q_outputs = self.encoder_q(feed_dict_q)  # queries: NxC
        
#         k,q = pair_embeddings(k_outputs, q_outputs, mode)
        k,q = k_outputs, q_outputs
        
        k = stack_features_across_batch(k, mode)
        q = stack_features_across_batch(q, mode)
        
        q = nn.functional.normalize(q, dim=1)
        
        with torch.no_grad():
            k = nn.functional.normalize(k, dim=1)
            
        with torch.no_grad():
            self.queue_view_ptr[0] = 0
            for feed_dict in feed_dicts_N:
                scene_negatives = self.forward(feed_dict_q=feed_dict, is_eval=True)
                self._dequeue_and_enqueue_view(scene_negatives)
            
        # compute logits
        # Einstein sum is more intuitive
        # positive logits: Nx1
        l_pos = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)
        # negative logits: Nxr
        l_neg = torch.einsum('nc,ck->nk', [q, self.queue_view.clone().detach()])

        # logits: Nx(1+r)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.T

        # labels: positive key indicators
        labels = torch.zeros(logits.shape[0], dtype=torch.long).cuda()

#         # dequeue and enqueue
#         self._dequeue_and_enqueue_view(k)
        
        return logits, labels, None, None

## Testing the Node MoCo model 

In [ ]:
model = MoCo(r=40, mode="node")
model = model.cuda()

In [ ]:
outputs = model(feed_dict_q_, feed_dict_k_, metadata_)

TypeError: 'NoneType' object is not iterable

In [ ]:
logits, labels, _, __ = outputs

In [ ]:
logits.shape, labels.shape

## Testing the spatial MoCo model 

In [ ]:
model = MoCo(r=40, mode="spatial")
model = model.cuda()

In [ ]:
outputs_ = model(feed_dict_q_, feed_dict_k_, metadata_)

## Testing the Pairing of Embeddings

In [ ]:
train_iter = iter(train_loader)

feed_dict_q_, feed_dict_k_, metadata_ = next(train_iter)
    
feed_dict_q_["images"] = feed_dict_q_["images"].cuda()
feed_dict_k_["images"] = feed_dict_k_["images"].cuda()

outputs = model(feed_dict_q_, feed_dict_k_, metadata_)